# 📊🎬 **Minimal viable product: Pipeline de preparação de dados para análise de avaliação de filmes e demais dados.**

## 🚀 **Sprint: Engenharia de dados**  

**Aluno:** Rafael Henrique da Costa  
**RA:** 4052025002012  
**Turma:** 40530010002_20260_01


## Objetivo

O objetivo desse MVP é preparar a base do IMDb para possibilitar análises consistentes das avaliações de filmes e de suas principais características, considerando apenas os títulos que atendem aos critérios definidos durante o processo de tratamento e curadoria da base.

## Perguntas a serem respondidas

**1- Quantos filmes possuem avaliação?**  

**2- Qual a nota média geral dos filmes?**

**3- Qual o filme mais bem avaliado?**

**4- Qual o filme com maior número de votos?**

**5- Quantos filmes têm nota acima de 8?**

**6- Quantos filmes têm menos de 100 votos?**

**7- Qual a distribuição das notas médias?**

**8- Top 10 filmes mais bem avaliados (mínimo X votos)**

**9- Top 10 filmes com mais votos**

**10- Filmes “cult”: nota alta, poucos votos**

**11- Filmes “populares”: muitos votos, nota mediana**

**12- Quais filmes Christopher Nolan dirigiu?**

**13- Quantos filmes cada diretor dirigiu?**

**14- Quais os 5 filmes mais bem avaliados do Christopher Nolan?**

## Detalhamento

### **1- Busca pelos dados**  
Os dados foram obtidos no portal oficial da base de dados do IMDb (https://datasets.imdbws.com/). Os arquivos utilizado foram:
- **name.basics.tsv.gz**: possui dados básicos de cada pessoa.
- **title.basics.tsv.gz**: possui dados básicos de filmes, documentários, programas de TV.
- **title.principals.tsv.gz**: possui dados de quais são os integrantes principais (diretor, fotógrafo, etc...)
- **title.ratings.tsv.gz**: possui dados de nota média e quantidade de votos de cada filme, documentário, programa de TV.

### **2- Coleta**

Os dados foram coletados e armazenados em um bucket S3 diretamente na plataforma da Amazon AWS. Posteriormente foi feito a conexão entre a conta Databricks e a plataforma Amazon AWS para manipular os dados utilizando o Apache Spark. Para isso, foi necessário configurar parâmetros de acesso e autênticação entre as plataformas e obter as chaves de acesso para conseguir o devido acesso ao armazenamento em nuvem da Amazon AWS.

### **3- Modelagem**
O esquema utilizado foi o Estrela com fato e dimensões:

**Fato**: Avaliação de filmes (fact_movies_ratings)  
**Dimensões**: Filmes e pessoas (dim_movie, dim_person)

### **Catálogo de dados - Data Warehouse de Avaliações de Filmes**

**Tabela Fato: fact_movies_ratings**

📌 Descrição

Tabela fato que armazena métricas agregadas de avaliação por filme.
Cada linha representa um filme com sua nota média e quantidade total de votos.

🎯 Grão (Granularidade)

1 linha por filme (tconst)

🔑 Chave Primária

tconst

🔗 Chaves Estrangeiras

tconst → dim_movie.tconst

📊 **Métricas**

| Coluna           | Tipo   | Descrição                           |
| ---------------- | ------ | ----------------------------------- |
| `averageRatings` | DOUBLE | Nota média do filme (escala 0–10)   |
| `numVotes`       | INT    | Quantidade total de votos recebidos |


📈 **Exemplos de Perguntas Respondidas**

Quais são os filmes mais bem avaliados?

Qual a distribuição das notas médias?

Qual o filme mais bem avaliado (desempate por votos)?

Qual a média global das avaliações?

<br> 

**Dimensão: dim_movie**
📌 Descrição

Dimensão que armazena informações descritivas dos filmes.

🎯 Grão

1 linha por filme (tconst)

🔑 Chave Primária

- tconst

📊 **Métricas**

| Coluna           | Tipo   | Descrição                    |
| ---------------- | ------ | ---------------------------- |
| `tconst`         | STRING | Identificador único do filme |
| `primaryTitle`   | STRING | Título principal do filme    |
| `originalTitle`  | STRING | Título original              |
| `startYear`      | INT    | Ano de lançamento            |
| `runtimeMinutes` | INT    | Duração do filme em minutos  |
| `genres`         | STRING | Gêneros do filme             |

📈 Exemplos de Perguntas Respondidas

**Filmes lançados por ano**

**Filmes por gênero**

**Filmes mais bem avaliados com título e ano**

**Análise histórica de avaliações**

<br>

**Dimensão: dim_person**

📌 Descrição

Dimensão que contém informações básicas das pessoas envolvidas nos filmes (diretores, atores, etc.).

🎯 Grão

1 linha por pessoa (nconst)

🔑 Chave Primária

- nconst

📊 **Métricas**
| Coluna        | Tipo   | Descrição                             |
| ------------- | ------ | ------------------------------------- |
| `nconst`      | STRING | Identificador único da pessoa         |
| `primaryName` | STRING | Nome principal                        |
| `birthYear`   | INT    | Ano de nascimento                     |
| `deathYear`   | INT    | Ano de falecimento (quando aplicável) |

📈 Exemplos de Perguntas Respondidas

**Quantos filmes cada diretor dirigiu**

**Filmes associados a uma pessoa**

**Ranking de diretores por quantidade de filmes**

**Ranking de diretores por avaliação média**

<br>

**Tabela ponte: bridge_movie_person**
| Coluna     | Descrição                              |
| ---------- | -------------------------------------- |
| `tconst`   | Filme                                  |
| `nconst`   | Pessoa                                 |
| `category` | Papel no filme (director, actor, etc.) |

### 4- Carga

O pipeline de dados deste projeto foi estruturado seguindo a arquitetura Medallion (Bronze, Silver e Gold), amplamente utilizada em ambientes de Data Lakehouse, com o objetivo de garantir qualidade, rastreabilidade, escalabilidade e clareza analítica ao longo do processamento.

🔹 **Camada Bronze — Ingestão dos Dados Brutos**

A camada Bronze é responsável pela ingestão dos dados em seu formato original, sem alterações estruturais ou semânticas.

Os arquivos da base do IMDb, originalmente no formato TSV, foram armazenados no Amazon S3.

Esses arquivos foram carregados no Databricks utilizando Apache Spark, preservando:

Tipos de dados originais (majoritariamente string)

Valores ausentes representados como \N (Na documentação do IMDb é detalhado que os valores nulos são representados como \N)

Possíveis duplicidades

Nenhuma limpeza ou transformação de negócio foi aplicada nesta etapa.

Objetivo da camada Bronze:
Garantir uma cópia fiel e auditável da fonte de dados original.

🔹 **Camada Silver — Limpeza, Padronização e Qualidade dos Dados**

A camada Silver é responsável por transformar os dados brutos em dados confiáveis e consistentes, prontos para modelagem analítica.

Principais etapas realizadas:

Tratamento de valores nulos:

Conversão explícita do valor \N para NULL, permitindo operações de cast e agregação.

Conversão de tipos de dados:

Colunas numéricas (ex.: ano, quantidade de votos, notas) foram convertidas para tipos numéricos adequados.

Filtragem de registros inválidos:

Remoção de registros sem identificadores essenciais ou atributos obrigatórios.

Remoção de duplicatas:

Garantia de unicidade de entidades como filmes e pessoas.

Normalização semântica:

Separação clara entre entidades (filmes, pessoas, avaliações, principals).

Ao final, os dados da camada Silver foram gravados em Delta Lake, permitindo versionamento, confiabilidade e melhor performance.

Objetivo da camada Silver:
Fornecer dados limpos, tipados e semanticamente corretos, prontos para modelagem dimensional.

🔹 **Camada Gold — Modelagem Dimensional e Dados Analíticos**

A camada Gold contém os dados finalmente modelados para análise, seguindo o modelo dimensional (estrela) e preparados para consultas SQL e visualizações.

Nesta camada foram criadas:

📌 Dimensões

dim_movie
Contém atributos descritivos dos filmes (título, ano, tipo, etc.).

dim_person
Contém informações básicas das pessoas envolvidas nos filmes (nome, datas, etc.).

📌 Tabela de Ponte (Bridge)

bridge_movie_person
Modela o relacionamento muitos-para-muitos entre filmes e pessoas, incluindo o papel exercido (diretor, ator, etc.).

Essa tabela não contém métricas, apenas relacionamentos.

📌 Fato

fact_movies_ratings
Contém as métricas analíticas principais:

Nota média do filme

Quantidade de votos

Essa modelagem permite análises como:

Avaliações por filme

Avaliações por diretor

Rankings e comparações

Análises agregadas sem duplicação de métricas

Todas as tabelas Gold foram gravadas em Delta Lake no S3 e registradas no metastore do Databricks, permitindo consultas SQL diretas via catálogo.

Objetivo da camada Gold:
Disponibilizar dados prontos para análise, com integridade dimensional e métricas confiáveis.

## 5- Análise

### 5a- Qualidade dos dados

Os dados do IMDb já vieram de certa forma bem curados, sem identificadores nulos, sem duplicidade, etc.. Apenas foi preciso alterar o identificador de nulidade \N por NULL (ou None). Mais a seguir nas células de tratamendo de dados Silver está o código do que foi feito para cada tabela.

### 5b- Solução do problema

Para a solução das perguntas definidas no objetivo do projeto foi utilizado o SQL para consultar a camada Gold dos dados e exibir o resultado em uma tabela que é retornada no próprio notebook do Databricks e, caso necessário, foi feita a utilização das bibliotecas Python Seaborn e Matplotlib para plotagem de gráficos para uma melhor visualização.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

s3_bucket = "s3a://imdb-mvp/"

spark = (SparkSession.builder .appName("Imdb-MVP") \
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
        .config("spark.hadoop.fs.s3a.access.key", dbutils.secrets.get(scope="imdb-mvp", key="AWS_ACCESS_KEY")) \
        .config("spark.hadoop.fs.s3a.secret.key", dbutils.secrets.get(scope="imdb-mvp", key="AWS_SECRET_ACCESS_KEY")) \
        .getOrCreate() )

In [0]:
name_df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "name.basics.tsv")

name_df.show(10)

In [0]:
name_df.filter(name_df.primaryName == "Christopher Nolan").show()

In [0]:
title_df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.basics.tsv")

title_df.show(10)

In [0]:
tronIds = [result["tconst"] for result in title_df.filter(title_df.originalTitle.like("Tron:%")).filter(title_df.titleType == "movie").select("tconst").collect()]

In [0]:
rating_df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.ratings.tsv")

rating_df.show(10)

In [0]:
from pyspark.sql import functions as F

title_filtered = title_df.select("tconst", "originalTitle")
ratingFiltered = rating_df.filter(rating_df.tconst.isin(tronIds))

titleWithRatings = (
    ratingFiltered
    .join(title_filtered, on="tconst", how="left")
)

titleWithRatings.show()



In [0]:
idCn = name_df.filter(name_df.primaryName == "Christopher Nolan").filter(name_df.primaryProfession.like("%director%")).filter(name_df.birthYear != "\\N").select("nconst").collect()[0]["nconst"]

print(idCn)

idCnTitles = [result["knownForTitles"] for result in name_df.filter(name_df.primaryName == "Christopher Nolan").filter(name_df.primaryProfession.like("%director%")).filter(name_df.birthYear != "\\N").select("knownForTitles").collect()]

splittedIds = idCnTitles[0].split(",")

print(splittedIds)

In [0]:
principals_df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.principals.tsv")

principals_df.show(10)

In [0]:
cnDirectedTitlesId = [result["tconst"] for result in principals_df.filter(principals_df.nconst == idCn).filter(principals_df.category == "director").select("tconst").collect()]

print(cnDirectedTitlesId)

In [0]:
cnTitles_df = title_df.filter(title_df.tconst.isin(cnDirectedTitlesId)).select("tconst", "originalTitle", "startYear", "genres")

In [0]:
filteredCnTitlesRatings = (
    rating_df.filter(rating_df.tconst.isin(cnDirectedTitlesId)).select("tconst", "averageRating")
)

cnTitlesRatings_df = (
    cnTitles_df
    .join(filteredCnTitlesRatings, on="tconst", how="left")
)

cnTitlesRatings_df = cnTitlesRatings_df.filter(cnTitlesRatings_df.averageRating.isNotNull()).sort(cnTitlesRatings_df.averageRating.desc())

cnTitlesRatings_df.show()


In [0]:
cnTitlesRatings_df.first()

In [0]:
titleBasicsBronze = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.basics.tsv")

titleBasicsBronze.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_bronze/title/basics")


In [0]:
titlePrincipalsBronze = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.principals.tsv")

titlePrincipalsBronze.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_bronze/title/principals")

In [0]:
titleRatingsBronze = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "title.ratings.tsv")

titleRatingsBronze.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_bronze/title/ratings")

In [0]:
nameBasicsBronze = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(s3_bucket + "name.basics.tsv")

nameBasicsBronze.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_bronze/name/basics")

In [0]:
bronzeTitlesBasicsRaw = spark.read.format("delta").load(s3_bucket + "imdb_bronze/title/basics")
bronzeTitlesBasics = bronzeTitlesBasicsRaw.replace("\\N", None)

In [0]:
from pyspark.sql import functions as F

total = bronzeTitlesBasics.count()

exprs = [
    F.count(F.when(F.col(c).isNull(), c)).alias(c + "_nulls")
    for c in bronzeTitlesBasics.columns
]

result = bronzeTitlesBasics.select(exprs)

# mostrar % também
for col in result.columns:
    nulls = result.collect()[0][col]
    print(f"{col}: {nulls} ({nulls/total*100:.2f}%)")

In [0]:
bronzeTitlesBasics.printSchema()

In [0]:
tbtest = bronzeTitlesBasics.filter(bronzeTitlesBasics.tconst == "tt25616472")
tbtest.show()

In [0]:
from pyspark.sql.functions import col, when

silverMoviesBasics = (
    bronzeTitlesBasics \
    .filter(bronzeTitlesBasics.titleType == "movie") \
    .withColumn("year", when(col("startYear") == "\\N", None) \
                .otherwise(col("startYear").cast("int"))) \
    .withColumn("runtimeMinutes", when(col("runtimeMinutes") == "\\N", None) \
                .otherwise(col("runtimeMinutes").cast("int")) \
                .cast("int")) \
    .select("tconst", "titleType", "primaryTitle", "originalTitle", "isAdult", "year", "runtimeMinutes", "genres")
)

silverMoviesBasics.show()

In [0]:
from pyspark.sql import functions as F

total = silverMoviesBasics.count()

exprs = [
    F.count(F.when(F.col(c).isNull(), c)).alias(c + "_nulls")
    for c in silverMoviesBasics.columns
]

result = silverMoviesBasics.select(exprs)

# mostrar % também
for col in result.columns:
    nulls = result.collect()[0][col]
    print(f"{col}: {nulls} ({nulls/total*100:.2f}%)")

In [0]:
silverMoviesBasics.printSchema()

In [0]:
silverMoviesBasics.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_silver/title/basics")

In [0]:
bronzeTitlesPrincipalsRaw = spark.read.format("delta").load(s3_bucket + "imdb_bronze/title/principals")
bronzeTitlesPrincipalsRaw.show()

In [0]:
bronzeTitlesPrincipalsRaw.printSchema()

In [0]:
silverTitlesPrincipals = bronzeTitlesPrincipalsRaw.replace("\\N", None).select("tconst", "ordering", "nconst", "category", "characters")
silverTitlesPrincipals.show()

In [0]:
silverTitlesPrincipals.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_silver/title/principals")

In [0]:
bronzeTitlesRatingsRaw = spark.read.format("delta").load(s3_bucket + "imdb_bronze/title/ratings")
bronzeTitlesRatingsRaw.show()

In [0]:
from pyspark.sql import functions as F

total = bronzeTitlesRatingsRaw.count()

exprs = [
    F.count(F.when(F.col(c).isNull(), c)).alias(c + "_nulls")
    for c in bronzeTitlesRatingsRaw.columns
]

result = bronzeTitlesRatingsRaw.select(exprs)

# mostrar % também
for col in result.columns:
    nulls = result.collect()[0][col]
    print(f"{col}: {nulls} ({nulls/total*100:.2f}%)")

In [0]:
silverTitlesRatings = bronzeTitlesRatingsRaw.replace("\\N", None)

In [0]:
silverTitlesRatings.printSchema()

In [0]:
silverTitlesRatings.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_silver/title/ratings")

In [0]:
bronzeNameBasicsRaw = spark.read.format("delta").load(s3_bucket + "imdb_bronze/name/basics")
bronzeNameBasicsRaw.show()

In [0]:
silverNameBasics = bronzeNameBasicsRaw.replace("\\N", None)

In [0]:
silverNameBasics.show()

In [0]:
silverNameBasics = silverNameBasics.withColumn("birthYear", silverNameBasics["birthYear"].cast("int"))
silverNameBasics = silverNameBasics.withColumn("deathYear", silverNameBasics["deathYear"].cast("int"))

In [0]:
silverNameBasics.printSchema()

In [0]:
silverNameBasics.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_silver/name/basics")

In [0]:
silverRatings = spark.read.format("delta").load(s3_bucket + "imdb_silver/title/ratings")
silverMoviesBasics = spark.read.format("delta").load(s3_bucket + "imdb_silver/title/basics")

In [0]:
factMoviesRatings = silverRatings.join(
    silverMoviesBasics.select("tconst"),
    on="tconst",
    how="inner"
)

In [0]:
factMoviesRatings.show()

In [0]:
silverNameBasics = spark.read.format("delta").load(s3_bucket + "imdb_silver/name/basics")

dimPerson = (
    silverNameBasics.select("nconst", "primaryName", "birthYear", "deathYear").dropDuplicates(["nconst"])
)

dimPerson.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_gold/dim/person")

In [0]:
silverTitleBasics = spark.read.format("delta").load(s3_bucket + "imdb_silver/title/basics")

dimMovie = (
    silverTitleBasics.select("tconst", "primaryTitle", "originalTitle", "isAdult", "year", "runtimeMinutes", "genres").dropDuplicates(["tconst"])
)

dimMovie.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_gold/dim/movie")

In [0]:
silverTitlePrincipals = spark.read.format("delta").load(s3_bucket + "imdb_silver/title/principals")

silverTitlePrincipals = silverTitlePrincipals.select("tconst", "nconst", "category").dropDuplicates(["tconst", "nconst"])

In [0]:
bridgeMoviePerson = silverTitlePrincipals.join(dimMovie.select("tconst"), on="tconst", how="inner").join(dimPerson.select("nconst"), on="nconst", how="inner")

In [0]:
bridgeMoviePerson.show()

In [0]:
bridgeMoviePerson.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_gold/bridge/movie-person")

In [0]:
silverRatings = spark.read.format("delta").load(s3_bucket + "imdb_silver/title/ratings")
dimMovie = spark.read.format("delta").load(s3_bucket + "imdb_gold/dim/movie")

factMoviesRatings = silverRatings.join(
    dimMovie.select("tconst"),
    on="tconst",
    how="inner"
)

In [0]:
factMoviesRatings.write.format("delta").mode("overwrite").save(s3_bucket + "imdb_gold/fact/movies-ratings")

In [0]:
%sql

CREATE DATABASE IF NOT EXISTS movies_ratings_gold;

CREATE TABLE IF NOT EXISTS movies_ratings_gold.dim_movie
USING DELTA
LOCATION 's3://imdb-mvp/imdb_gold/dim/movie';

CREATE TABLE IF NOT EXISTS  movies_ratings_gold.dim_person
USING DELTA
LOCATION 's3://imdb-mvp/imdb_gold/dim/person';

CREATE TABLE IF NOT EXISTS  movies_ratings_gold.bridge_movie_person
USING DELTA
LOCATION 's3://imdb-mvp/imdb_gold/bridge/movie-person';

CREATE TABLE IF NOT EXISTS  movies_ratings_gold.fact_movies_ratings
USING DELTA
LOCATION 's3://imdb-mvp/imdb_gold/fact/movies-ratings';

In [0]:
%sql
SELECT DISTINCT

m.originalTitle,
m.year 
FROM movies_ratings_gold.bridge_movie_person b 
JOIN movies_ratings_gold.dim_person p 
  ON b.nconst = p.nconst
JOIN movies_ratings_gold.dim_movie m 
  ON b.tconst = m.tconst
WHERE p.primaryName = "Christopher Nolan"
  AND b.category = "director"
ORDER BY m.year DESC

In [0]:
%sql
SELECT DISTINCT
m.originalTitle AS title,
m.year AS release_year,
f.averageRating AS average_rating,
f.numVotes AS votes_quantity

FROM movies_ratings_gold.bridge_movie_person b
JOIN movies_ratings_gold.dim_person p
  ON b.nconst = p.nconst
JOIN movies_ratings_gold.dim_movie m
  ON b.tconst = m.tconst
JOIN movies_ratings_gold.fact_movies_ratings f
  ON b.tconst = f.tconst
WHERE p.primaryName = "Christopher Nolan"
  AND b.category = "director"
ORDER BY average_rating DESC;

In [0]:
%sql
SELECT DISTINCT
m.originalTitle AS title,
m.year AS release_year,
f.averageRating AS average_rating,
f.numVotes AS votes_quantity

FROM movies_ratings_gold.bridge_movie_person b
JOIN movies_ratings_gold.dim_person p
  ON b.nconst = p.nconst
JOIN movies_ratings_gold.dim_movie m
  ON b.tconst = m.tconst
JOIN movies_ratings_gold.fact_movies_ratings f
  ON b.tconst = f.tconst
WHERE p.primaryName = "Christopher Nolan"
  AND b.category = "director"
ORDER BY average_rating DESC
LIMIT 5;

In [0]:
# QUAIS OS 5 FILMES MAIS BEM AVALIADOS DIRIGIDOS POR CHRISTOPHER NOLAN?

sdf = spark.sql("""SELECT DISTINCT
m.originalTitle AS title,
m.year AS release_year,
f.averageRating AS average_rating,
f.numVotes AS votes_quantity

FROM movies_ratings_gold.bridge_movie_person b
JOIN movies_ratings_gold.dim_person p
  ON b.nconst = p.nconst
JOIN movies_ratings_gold.dim_movie m
  ON b.tconst = m.tconst
JOIN movies_ratings_gold.fact_movies_ratings f
  ON b.tconst = f.tconst
WHERE p.primaryName = "Christopher Nolan"
  AND b.category = "director"
ORDER BY average_rating DESC
LIMIT 5;""")

df = sdf.toPandas()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.stripplot(
    data=df,
    x="title",
    y="average_rating",
    size=7,
    color="orange",
    edgecolor="auto",
    linewidth=1
)

plt.xlabel("Movie title")
plt.ylabel("Average rating")
plt.title("Top 5 best movies by Christopher Nolan")

plt.show()

In [0]:
%sql

/* QUAL A QUANTIDADE DE FILMES NA DIMENSÃO FILME? */

SELECT COUNT(*) as movies_qty
FROM movies_ratings_gold.dim_movie

In [0]:
%sql

/* QUAL A QUANTIDADE DE FILMES QUE TEM AVALIAÇÃO? */

SELECT COUNT(*) as movies_rating_qty
FROM movies_ratings_gold.fact_movies_ratings

In [0]:
%sql

/* QUAL A MÉDIA GLOBAL DE AVALIAÇÕES? */

CREATE VIEW IF NOT EXISTS global_average_ratings AS
SELECT SUM(f.averageRating * f.numVotes) / SUM(f.numVotes) AS global_average
FROM movies_ratings_gold.fact_movies_ratings f;

SELECT * FROM global_average_ratings;

In [0]:
%sql

/* QUEL O FILME MAIS BEM AVALIADO? (NOTA MÉDIA USANDO NÚMERO DE VOTOS COMO DESEMPATE) */

SELECT
m.originalTitle AS title,
f.averageRating AS average_rating,
f.numVotes AS votes_qty

FROM movies_ratings_gold.fact_movies_ratings f
JOIN movies_ratings_gold.dim_movie m
  ON f.tconst = m.tconst
ORDER BY f.averageRating DESC, f.numVotes DESC
LIMIT 1;

In [0]:
%sql
SELECT

/* QUAL O FILME COM O MAIOR NÚMERO DE VOTOS? */

m.originalTitle AS title,
f.numVotes AS votes_qty

FROM movies_ratings_gold.fact_movies_ratings f 
JOIN movies_ratings_gold.dim_movie m 
  ON f.tconst = m.tconst
ORDER BY f.numVotes DESC
LIMIT 1;

In [0]:
%sql

/* QUANTOS FILMES TEM NOTA ACIMA DE 8? */

SELECT
COUNT(*) AS movies_qty

FROM movies_ratings_gold.fact_movies_ratings f
WHERE f.averageRating > 8.0;

In [0]:
%sql

/* QUANTOS FILMES TEM MENOS DE 100 VOTOS? */

SELECT
COUNT(*) AS movies_qty

FROM movies_ratings_gold.fact_movies_ratings f 
WHERE f.numVotes < 100

In [0]:
%sql
SELECT
FLOOR(f.averageRating) AS rating_interval,
COUNT(*) AS movies_qty

FROM movies_ratings_gold.fact_movies_ratings f 
WHERE f.numVotes > 1000
GROUP BY FLOOR(f.averageRating)
ORDER BY rating_interval;

In [0]:
sdf = spark.sql("""
                SELECT f.averageRating AS average_rating
                FROM movies_ratings_gold.fact_movies_ratings f 
                WHERE f.numVotes > 1000
                """)

df = sdf.toPandas()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(
    df["average_rating"],
    bins=20,
    kde=True
)

plt.xlabel("Average Rating")
plt.ylabel("Movies quantity")
plt.title("Movies average rating distribution")
plt.show()

In [0]:
%sql

/* QUAL O FILME COM A PIOR AVALIAÇÃO? */

SELECT
m.originalTitle AS title,
f.averageRating AS average_rating,
f.numVotes AS votes_qty
FROM movies_ratings_gold.fact_movies_ratings f
JOIN movies_ratings_gold.dim_movie m
ON f.tconst = m.tconst
ORDER BY f.averageRating ASC, f.numVotes DESC
LIMIT 1;

In [0]:
# QUAIS OS 10 FILMES MAIS BEM AVALIADOS QUE TENHAM MAIS DE 1000 VOTOS?

df = spark.sql("""
                SELECT
                m.originalTitle AS title,
                f.averageRating AS average_rating,
                f.numVotes AS num_votes
                FROM movies_ratings_gold.fact_movies_ratings f
                JOIN movies_ratings_gold.dim_movie m
                ON f.tconst = m.tconst
                WHERE f.numVotes > 1000
                ORDER BY f.averageRating DESC, f.numVotes DESC
                LIMIT 10;
                """).toPandas()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

plot = sns.barplot(
    df,
    x="average_rating",
    y="title",
)
plot.bar_label(plot.containers[0], fontsize=10)

plt.xlabel("Average Rating")
plt.ylabel("Movie Title")
plt.title("Top 10 movies with more than 1K votes")

In [0]:
# Quais os 10 filmes com maior número de votos?

df = spark.sql("""
               SELECT
               m.originalTitle AS title,
               f.numVotes AS votes_qty
               FROM movies_ratings_gold.fact_movies_ratings f 
               JOIN movies_ratings_gold.dim_movie m 
               ON f.tconst = m.tconst
               ORDER BY f.numVotes DESC
               LIMIT 10;
               """).toPandas()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def millions(x, pos):
    return f'{x / 1e6:.2f}M'

plt.figure(figsize=(14, 6))
plot = sns.barplot(
    df,
    x="votes_qty",
    y="title",
)

formatter = FuncFormatter(millions)
for container in plot.containers:
    plot.bar_label(container, fmt=formatter, padding=3, fontsize=10)

plot.xaxis.set_major_formatter(formatter)

plt.xlabel("Number of votes")
plt.ylabel("Movie title")
plt.title("Top 10 most voted movies")

In [0]:
%sql

/* QUAIS OS FILMES MAIS "CULT" (NOTA ALTA E POUCOS VOTOS)? */

SELECT
m.tconst AS movie_id,
m.originalTitle AS title,
f.averageRating AS average_rating,
f.numVotes AS votes_qty

FROM movies_ratings_gold.fact_movies_ratings f 
JOIN movies_ratings_gold.dim_movie m
ON f.tconst = m.tconst
WHERE f.numVotes > 10000 AND f.numVotes < 30000
ORDER BY f.averageRating DESC, f.numVotes ASC
LIMIT 10;


In [0]:
# QUAIS OS 10 FILMES MAIS POPULARES (MUITOS VOTOS E NOTA MEDIANA)?

df = spark.sql("""
               SELECT
               m.originalTitle AS title,
               f.averageRating AS average_rating,
               f.numVotes AS votes_qty

               FROM movies_ratings_gold.fact_movies_ratings f 
               JOIN movies_ratings_gold.dim_movie m 
               ON f.tconst = m.tconst
               WHERE f.averageRating > 5.5 AND f.averageRating < 7.6
               ORDER BY f.numVotes DESC
               LIMIT 10;
               """).toPandas()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def millions(x, pos):
    return f'{x / 1e6:.1f}M'

plot = sns.barplot(
    df,
    x="votes_qty",
    y="title",
    hue="average_rating"
)

formatter = FuncFormatter(millions)
plot.xaxis.set_major_formatter(formatter)

plt.xlabel("Votes quantity")
plt.ylabel("Movie title")
plt.title("Top 10 most popular movies")

In [0]:
%sql

/* EM QUAIS FILMES O CHRISTOPHER NOLAN FOI DIRETOR? */

SELECT
m.originalTitle AS title,
f.averageRating AS average_rating,
f.numVotes AS votes_qty,
p.primaryName AS director

FROM movies_ratings_gold.fact_movies_ratings f 
JOIN movies_ratings_gold.bridge_movie_person b 
ON f.tconst = b.tconst
JOIN movies_ratings_gold.dim_movie m 
ON b.tconst = m.tconst
JOIN movies_ratings_gold.dim_person p 
ON b.nconst = p.nconst
WHERE p.primaryName = "Christopher Nolan" AND b.category = "director";

In [0]:
%sql

/* QUANTOS FILMES CADA DIRETOR DIRIGIU? */

SELECT
p.primaryName AS director,
COUNT(DISTINCT b.tconst) AS movies_qty

FROM movies_ratings_gold.bridge_movie_person b 
JOIN movies_ratings_gold.dim_person p 
ON b.nconst = p.nconst
JOIN movies_ratings_gold.dim_movie m 
ON b.tconst = m.tconst
WHERE b.category = "director"
GROUP BY p.primaryName
ORDER BY COUNT(b.tconst) DESC